In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Setting up the environment

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 8.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 23.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 833.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00:00

In [ ]:
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps git+https://github.com/huggingface/transformers.git # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

In [ ]:
!pip install huggingface-hub==0.34.0
!pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.7/558.7 kB 9.8 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.34.3
    Uninstalling huggingface-hub-0.34.3:
      Successfully uninstalled huggingface-hub-0.34.3


## Loading the unsloth gemma model

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit", # Or "unsloth/gemma-3n-E2B-it"
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-05 23:07:13.220506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754435233.571809      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754435233.670600      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Gemma3N patching. Transformers: 4.56.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [ ]:
# Common imports
import os
import optax
import treescope

# imports and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datasets import Dataset

## Data Preparation

In [ ]:
# Login using e.g. `huggingface-cli login` to access this dataset
data = pd.read_csv("hf://datasets/infinite-dataset-hub/PregnancyBotConvo/data.csv",index_col=None)
data = data[["user_input", "bot_response"]].dropna()
data

,user_input,bot_response
0,What are some good exercises during pregnancy?,Certain exercises can help maintain fitness an...
1,I'm worried about morning sickness. Any tips?,There are several strategies to help manage mo...
2,What should I know about prenatal vitamins?,Prenatal vitamins are crucial for your baby's ...
3,How can I prepare for a natural birth?,Preparing for a natural birth involves underst...
4,I'm feeling anxious about childbirth. What sho...,Feeling anxious about childbirth is common. It...
...,...,...
94,What if something goes wrong during the birth?,It's natural to worry about complications duri...
95,Can I practice prenatal yoga?,Prenatal yoga is specifically designed to acco...
96,Should I avoid certain smells to help with mor...,"Yes, certain smells can trigger nausea. Try to..."
97,Can I take prenatal vitamins if I'm not pregnant?,Taking prenatal vitamins is not only for pregn...


In [ ]:
final_test = data.iloc[96:]
test

[{'text': "<bos><start_of_turn>user\nMy partner and I are worried about the baby's health. How can we stay calm?<end_of_turn>\n<start_of_turn>model\nIt's natural to feel worried. Consider reading about common pregnancy complications and symptoms, and remember that most babies are born healthy. It's important to communicate openly with your partner and healthcare provider.<end_of_turn>\n"},
 {'text': '<bos><start_of_turn>user\nAre prenatal vitamins necessary for everyone?<end_of_turn>\n<start_of_turn>model\nWhile prenatal vitamins can be beneficial for most pregnant individuals, they are not a one-size-fits-all solution. Your healthcare provider can help determine if prenatal vitamins are necessary based on your individual health and dietary needs.<end_of_turn>\n'},
 {'text': "<bos><start_of_turn>user\nWhat should I do if I'm experiencing extreme morning sickness?<end_of_turn>\n<start_of_turn>model\nExtreme morning sickness, also known as hyperemesis gravidarum, may require medical inte

In [ ]:
data.drop(final_test.index, inplace = True)
data

,user_input,bot_response
0,What are some good exercises during pregnancy?,Certain exercises can help maintain fitness an...
1,I'm worried about morning sickness. Any tips?,There are several strategies to help manage mo...
2,What should I know about prenatal vitamins?,Prenatal vitamins are crucial for your baby's ...
3,How can I prepare for a natural birth?,Preparing for a natural birth involves underst...
4,I'm feeling anxious about childbirth. What sho...,Feeling anxious about childbirth is common. It...
...,...,...
91,What are the signs of a healthy pregnancy?,Signs of a healthy pregnancy include regular p...
92,Are prenatal vitamins necessary for everyone?,While prenatal vitamins can be beneficial for ...
93,What's the best way to manage labor pain?,"There are several pain management options, inc..."
94,What if something goes wrong during the birth?,It's natural to worry about complications duri...


In [ ]:
# defining the tokenizer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
# This function formats a row of dialogue history using the tokenizer's chat template.
# It expects each row to have 'user_input' and 'bot_response' keys.
# The output is a single string prompt formatted for chat-based language models.
def format_prompt(row):
    return tokenizer.apply_chat_template([
        {"role": "user", "content": row["user_input"]},
        {"role": "assistant", "content": row["bot_response"]},
    ])


In [ ]:
# applying the function to the prompt
data["chat_prompt"] = data.apply(format_prompt, axis=1)

In [ ]:
# shuffling the data
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
dataset = data[['chat_prompt']]

# splitting the train and test
train = dataset[:93]
test = dataset[93:]
dataset = [{"text": row} for row in dataset["chat_prompt"].tolist()]
train = [{"text": row} for row in train["chat_prompt"].tolist()]
test = [{"text": row} for row in test["chat_prompt"].tolist()]
dataset

[{'text': "<bos><start_of_turn>user\nWhat's the best time of day to exercise during pregnancy?<end_of_turn>\n<start_of_turn>model\nExercise at any time of day that suits you best. However, try to maintain a consistent schedule and avoid extreme temperatures, especially during the first trimester.<end_of_turn>\n"},
 {'text': "<bos><start_of_turn>user\nAre prenatal vitamins safe during breastfeeding?<end_of_turn>\n<start_of_turn>model\nPrenatal vitamins are safe during breastfeeding, but you should continue to consult with your healthcare provider to ensure you're meeting your nutritional needs.<end_of_turn>\n"},
 {'text': '<bos><start_of_turn>user\nWhat are the signs of labor starting?<end_of_turn>\n<start_of_turn>model\nSigns of labor starting include regular contractions, lower back pain, rupture of membranes, and a bloody show. Contact your healthcare provider when you suspect labor has begun.<end_of_turn>\n'},
 {'text': "<bos><start_of_turn>user\nWhat if something goes wrong during 

In [ ]:
# converting tho dataset
from datasets import Dataset
train_dataset = Dataset.from_list(train)
eval_dataset = Dataset.from_list(test)

## Modelling

In [ ]:
# hyperparameters
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # Should leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [ ]:
# training configurations
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    use_cache = False,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 75,       learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",

        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/93 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
# Import the helper function from Unsloth to modify the trainer's formatting behavior
from unsloth.chat_templates import train_on_responses_only

# Reconfigure the trainer to train only on the assistant's (model's) responses,
# ignoring the user input when calculating loss. This is useful for fine-tuning
# where we want the model to learn to generate helpful replies.
trainer = train_on_responses_only(
    trainer,  # The original Trainer object from Hugging Face
    instruction_part = "<start_of_turn>user\n",   # Marks the beginning of the user prompt
    response_part    = "<start_of_turn>model\n",  # Marks the beginning of the assistant's response
)


Map (num_proc=4):   0%|          | 0/93 [00:00<?, ? examples/s]

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Map (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
# viewing one row of the datatset
tokenizer.decode(trainer.train_dataset[10]["input_ids"])

"<bos><bos><start_of_turn>user\nI'm worried about the changes my body will go through during pregnancy. What can I do?<end_of_turn>\n<start_of_turn>model\nIt's natural to feel anxious about physical changes during pregnancy. Stay informed about what to expect, maintain a healthy lifestyle, and remember that your body is doing an amazing job.<end_of_turn>\n"

In [ ]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 93
})

In [ ]:
# @title Show current memory stats
import gc
gc.collect()
torch.cuda.empty_cache()
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
10.988 GB of memory reserved.


In [ ]:
#for memory conservation
import torch
torch._dynamo.config.cache_size_limit = 128  # or 256, or 512 depending on your RAM


In [ ]:
# training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 93 | Num Epochs = 4 | Total steps = 75
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 10,567,680 of 5,450,005,952 (0.19% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,9.636200
2,8.805900
3,9.066600
4,9.549700
5,11.354500
6,10.471600
7,10.450900
8,10.001900
9,11.129600
10,10.481000


In [ ]:
#@title Show final memory and time stats
gc.collect()
torch.cuda.empty_cache()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

414.9291 seconds used for training.
6.92 minutes used for training.
Peak reserved memory = 10.988 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 74.54 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": "How do I detect signs of postpartum complications?"}]
    }
]


In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
final = tokenizer.batch_decode(outputs)
# Decode and return
#, skip_special_tokens=True)[0].strip()
final


['<bos><start_of_turn>user\nHow do I detect signs of postpartum complications?<end_of_turn>\n<start_of_turn>model\nYou should be aware of signs of postpartum complications like infection, bleeding, or excessive swelling. Contact your healthcare provider immediately if you notice any of these symptoms.<end_of_turn>']

In [ ]:
from unsloth.chat_templates import get_chat_template

def generate_response(model, tokenizer, user_message: str, device: str = "cuda") -> str:
    """
    Generates a response from the model using a structured chat message.

    Parameters:
    -----------
    model : torch.nn.Module
        The language model used for text generation (e.g., Gemma 3N via Unsloth).

    tokenizer : transformers.PreTrainedTokenizer
        The tokenizer compatible with the model, patched using Unsloth's `get_chat_template`.

    user_message : str
        The user's input or question to generate a response for.

    device : str
        The device to run inference on. Default is "cuda".

    Returns:
    --------
    str : The generated text response from the model.
    """

    # Step 1: Format the message using OpenAI-style JSON
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": user_message}]
        }
    ]

    # Step 2: Ensure tokenizer has the right chat template for Gemma-3
    tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")

    # Step 3: Apply chat template and tokenize input
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,  # Add BOS tokens as needed
        return_tensors="pt",
        tokenize=True,
        return_dict=True,
    ).to(device)

    # Step 4: Generate text with model
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=1.0,
        top_p=0.95,
        top_k=64
    )

    # Step 5: Decode and return string (skipping special tokens)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    return decoded


In [ ]:
final_test

,user_input,bot_response
96,Should I avoid certain smells to help with mor...,"Yes, certain smells can trigger nausea. Try to..."
97,Can I take prenatal vitamins if I'm not pregnant?,Taking prenatal vitamins is not only for pregn...
98,I'm worried about postpartum depression. How c...,Educating yourself about postpartum depression...


In [ ]:
row = final_test['user_input'][96]
generate_response(model, tokenizer, user_message = row)

"user\nShould I avoid certain smells to help with morning sickness?\nmodel\nSome smells, like citrus or mint, may help alleviate nausea. However, it's important to avoid strong or unpleasant odors that might worsen your symptoms."

In [ ]:
row = final_test['user_input'][97]
generate_response(model, tokenizer, user_message = row)

"user\nCan I take prenatal vitamins if I'm not pregnant?\nmodel\nYes, prenatal vitamins contain essential nutrients like folic acid, iron, and calcium that are important for overall health, even if you're not pregnant. However, it's best to discuss your vitamin needs with your healthcare provider."

In [ ]:
row = final_test['user_input'][98]
generate_response(model, tokenizer, user_message = row)

"user\nI'm worried about postpartum depression. How can I prepare for it?\nmodel\nWhile you can't fully prepare for postpartum depression, it's helpful to educate yourself about the condition, its symptoms, and available resources. Talking to your healthcare provider about your concerns can also help you feel more prepared and supported."

In [ ]:
generate_response(model, tokenizer, user_message:

## Saving

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

In [ ]:
if True:
    model.push_to_hub_merged(
        "TechBuz/gemma_3n_chatbot_Nestie",
        tokenizer,
        token = secret_value_0
    )

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...a_3n_chatbot_Nestie/tokenizer.model:   1%|1         | 59.5kB / 4.70MB            

  ...ma_3n_chatbot_Nestie/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00003.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00003-of-00003.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ie/model-00003-of-00003.safetensors:   1%|          | 25.1MB / 2.82GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 3/3 [03:20<00:00, 66.91s/it]


In [ ]:
model.push_to_hub("TechBuz/gemma-3n_nestie_head", token = secret_value_0) # Online saving
tokenizer.push_to_hub("TechBuz/gemma-3n_nestie_head", token = secret_value_0) # Online saving

README.md:   0%|          | 0.00/604 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pjl9olncz/adapter_model.safetensors:  79%|#######9  | 33.5MB / 42.3MB            

Saved model to https://huggingface.co/TechBuz/gemma-3n_nestie_head


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp0uljbb84/tokenizer.model      : 100%|##########| 4.70MB / 4.70MB            

  /tmp/tmp0uljbb84/tokenizer.json       : 100%|##########| 33.4MB / 33.4MB            